# Using the ArcGIS API for Python to Manage Workforce Projects
In the following example, a Workforce Project will be configured to manage restaurant inspections.

### Import the required modules
This example requires version 1.4.1 or higher of the ArcGIS API for Python.

In [ ]:
import datetime
import json
import arcgis
from arcgis.apps import workforce
from arcgis.gis import GIS
from arcgis.geometry import Geometry

### Connect to ArcGIS Online and get the Project
Let's sign into ArcGIS Online and get the Workforce Project to manage.

In [ ]:
gis = GIS("https://arcgis.com", "workforce_backoffice")
item = gis.content.get("a6a82fcf1c264694890ed7b82aa2df2e")
project = workforce.Project(item)

### Check out the project schema
Let's look at the project json structure. Notice there are 2 webmaps, 4 feature services, a group, a folder, and some other metadata.

In [ ]:
project_data = item.get_data()
print(json.dumps(project_data, indent=2))

### Create an assignment type
Let's add a new assignment type named "Restaurant Inspection" to the project.

In [ ]:
restaurant_inspection = project.assignment_types.add(name="Restaurant Inspections")

### Add three workers to the project
Let's add three inspectors to the project.

In [ ]:
aaron = project.workers.add(name="Aaron Pulver",
                    user_id="aaron_nitro",
                    status="not_working")

james = project.workers.add(name="Jame McManus",
                    user_id="james_Nitro",
                    status="not_working")

nitropublisher = project.workers.add(name="Nitro Publisher",
                    user_id="nitropublisher",
                    status="not_working")

### View the restaurants that need an inspection
Let's view the restaurants that need to be inspected.

In [ ]:
restaurants_layer = gis.content.get("965b5702333742bd98bc5f0c0d4d1f69").layers[0]
webmap = gis.map("San Diego", zoomlevel=14)
webmap.add_layer(restaurants_layer)
webmap

### View the city districts
Let's view the different city districts.

In [ ]:
districts_layer = gis.content.get("08eecb2f46c740aaba316cef8a719a88").layers[0]
webmap = gis.map("San Diego", zoomlevel=12)
webmap.add_layer(districts_layer)
webmap

### View the districts data
Let's see what data is available. Notice the district number.

In [ ]:
districts_df = districts_layer.query().df
districts_df

### Add layers to dispatcher webmap
Let's add the restaurants and district layers to the webmap but make them not visible by default.

In [ ]:
project.dispatcher_webmap.remove_layer(project.dispatcher_webmap.layers[0])
project.dispatcher_webmap.remove_layer(project.dispatcher_webmap.layers[0])
project.dispatcher_webmap.add_layer(districts_layer, {"visibility": False})
project.dispatcher_webmap.add_layer(restaurants_layer, {"visibility": False})
project.dispatcher_webmap.add_layer(project.assignments_layer)
project.dispatcher_webmap.add_layer(project.workers_layer)
project.dispatcher_webmap.update({})

### Create assignments for each restaurant and assign based on district
Let's create an assignment to inspect each restaurants. If the restaurant is in district 7 then assign it to Aaron. If the restaurant is in distrit 8 then assign it to James. If the restaurant is in district 6, assign it to Nitro Publisher.

In [ ]:
# List of inspections to add
inspections = []

# dispatcher to create assignments
dispatcher = project.dispatchers.get(user_id=gis.users.me.username)

# Query all of the restaurants
restaurant_features = restaurants_layer.query(out_sr=3857).features
for restaurant in restaurant_features:
    
    # Determine which district (if any) the restaurant is in
    contains_df = districts_df.contains(Geometry(restaurant.geometry))
    container = districts_df[contains_df]

    # If it is in a district then lets make an assignment for it
    if not container.empty:
        district = container['district'].iloc[0]
        if district == 7:
            worker = aaron
            status = "assigned"
        elif district == 8:
            worker = james
            status = "assigned"
        elif district == 6:
            worker = nitropublisher
            status = "assigned"
        else:
            worker = None
            status = "unassigned"

        # Create the assignment
        inspections.append(workforce.Assignment(
            project,
            geometry=restaurant.geometry,
            location=restaurant.attributes['SanDiegoABC_Site_Addr1'],
            status=status,
            assignment_type=restaurant_inspection,
            worker=worker,
            dispatcher=dispatcher,
            priority="high",
            due_date=datetime.datetime.utcnow() + datetime.timedelta(days=3),
            assigned_date=datetime.datetime.utcnow()
        ))
        
# Add the assignments to your project
project.assignments.batch_add(inspections)

### View assignments
Let's validate that the assignments were created.

In [ ]:
webmap = gis.map("San Diego", zoomlevel=12)
webmap.add_layer(project.assignments_layer)
webmap

### Reset the project
For Demo Only - Use this to reset the project to re-run the entire notebook.

In [ ]:
project.assignments_item.layers[0].delete_features(where="1=1")
project.workers.batch_delete(project.workers.search())
project.assignment_types.batch_delete(project.assignment_types.search())
for layer in project.dispatcher_webmap.layers[:-2]:
    project.dispatcher_webmap.remove_layer(layer)
project.dispatcher_webmap.update({})